In [40]:
#################################################################################################
###Feature selection  &  methods test                                                         ###
###training & test dataset base on wangs:  functional_experimental_data_have_all_score_clean  ###
#################################################################################################

In [41]:
#import & dataread

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import svm,datasets
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import roc_curve, auc
from sklearn.decomposition import PCA

from pandas import Series,DataFrame
import itertools
from sklearn import metrics
from matplotlib import pyplot
import random

data = pd.read_csv('functional_experimental_data_have_all_score_clean.tsv', sep='\t')
#print(data.head())
baseset = DataFrame(data)
scorename = ['CHASMplus', 'FATHMM_Cancer_score', 'ParsSNP', 'CanDrA_Score', 'REVEL_score','SIFT_score','LRT_score','MutationAssessor_score','DANN_score']
f_score = baseset.loc[:,['CHASMplus', 'FATHMM_Cancer_score', 'ParsSNP', 'CanDrA_Score', 'REVEL_score','SIFT_score','LRT_score','MutationAssessor_score','DANN_score']]
label = baseset['label']
print(f_score)


      CHASMplus  FATHMM_Cancer_score   ParsSNP  CanDrA_Score  REVEL_score  \
0         0.632                -4.05  0.379535     13.060500        0.947   
1         0.515                -2.28  0.648592     13.050100        0.956   
2         0.424                -1.95  0.344255     12.950300        0.461   
3         0.221                -1.45  0.186465     12.528200        0.641   
4         0.431                -1.95  0.201420     12.997700        0.614   
5         0.648                -1.36  0.206577      4.719800        0.625   
6         0.406                -1.86  0.122573      2.105400        0.636   
7         0.304                -1.07  0.106733      1.760200        0.471   
8         0.566                -3.69  0.111447      3.344800        0.977   
9         0.458                -2.07  0.060360     10.519200        0.540   
10        0.921                -2.72  0.183283     10.630000        0.906   
11        0.042                 2.96  0.045029     -0.242150        0.546   

In [38]:
                            ####multiple imputation & simple imputation###
#####  SimpleImputer by  Scikit-learn Preprocessing data

#functional_experimental_data_have_all_score_clean need not imputation because not Missing values
#the function for ohter works

#functional_experimental_data_have_all_score_clean_missing.tsv has missing values .it is only a test!
import numpy as np
#from sklearn.impute import SimpleImputer
from statsmodels.imputation import mice

def new_missing_tab(tab):   #new a tab with random missing value
    data = pd.read_csv(tab, sep='\t')
    f1 = DataFrame(data)
    scorename = ['CHASMplus', 'FATHMM_Cancer_score', 'ParsSNP', 'CanDrA_Score', 'REVEL_score','SIFT_score','LRT_score','MutationAssessor_score','DANN_score']
    f2 = f1.loc[:,['CHASMplus', 'FATHMM_Cancer_score', 'ParsSNP', 'CanDrA_Score', 'REVEL_score','SIFT_score','LRT_score','MutationAssessor_score','DANN_score','label']]
    for index, row in f2.iterrows():
        if random.randint(0,9) == 1:
            f2.iloc[index,0] = "NaN"
    f2.to_csv('functional_experimental_data_have_all_score_clean_missing.csv',index=0)
            
def simple_imputer(miss_tab,strategy):
    data = pd.read_csv(miss_tab, sep=',')
    f2 = open("imp_test.txt",'w')
    f1 = DataFrame(data)
    column_name = ['CHASMplus', 'FATHMM_Cancer_score', 'ParsSNP', 'CanDrA_Score', 'REVEL_score','SIFT_score','LRT_score','MutationAssessor_score','DANN_score','label']
    for i in column_name:
        f2.write(str(i))
        f2.write("	")
    f2.write('\n')    
        
    chs = f1['CHASMplus']
    #print(chs[0:100])
    imp = SimpleImputer(missing_values = np.nan, strategy = strategy)
    imp.fit(f1)
    result = imp.transform(f1)
    
    
    for i in result:
        for j in i:
            f2.write(str(j))
            f2.write("	")
        f2.write('\n')
    
           

        
    #print(f2)
    
    
        

tab = 'functional_experimental_data_have_all_score_clean.tsv'

#####################new a tab with random missing value################################
#new_missing_tab(tab)  
########################################################################################

miss_table = 'functional_experimental_data_have_all_score_clean_missing.csv'

strategy = 'mean'  # strategy can be mean, median or most frequent
#simple_imputer(miss_tab,strategy)
####################################################
#multiple imputation



In [6]:
f = open('feature_importance.txt','r')
l = f.readline()
l = f.readline()
imp = []
while l:
    a = l.split('  ')
    imp.append(a[0])
    l = f.readline()
target = [0, 1, 2, 3, 4, 5, 7, 8, 9, 11, 14, 15, 19, 21, 26]
methods_list = []
for n in target:
    methods_list.append(imp[n])
print(methods_list,len(methods_list))

['MutPred_score', 'FATHMM_score', 'MetaLR_score', 'REVEL_score', 'MetaSVM_score', 'M-CAP_score', 'phyloP100way_vertebrate', 'MutationAssessor_score', 'GERP++_RS', 'VEST3_score', 'PROVEAN_score', 'GM12878_fitCons_score', 'H1-hESC_fitCons_score', 'phyloP20way_mammalian', 'phastCons20way_mammalian'] 15


In [ ]:
                            ####model value & data normalization###
#auc roc
def roc(label,score1,score2):
    fpr1, tpr1, _ = roc_curve(label, score1)
    roc_auc1 = auc(fpr1, tpr1)
    fpr2, tpr2, _ = roc_curve(label, score2)
    roc_auc2 = auc(fpr2, tpr2)
    plt.figure()
    plt.plot(fpr1, tpr1, color='navy', lw=2, label='chasm+ ROC curve (area = %0.2f)' % roc_auc1)
    plt.plot(fpr2, tpr2, color='red', lw=2, label='new score ROC curve (area = %0.2f)' % roc_auc2)
    #plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([-0.01, 1])
    plt.ylim([0, 1.01])
    plt.xlabel('False Positive Rate') 
    plt.ylabel('True Positive Rate')
    plt.title('chasm+ vs chasm+ impute missing value')
    plt.legend(loc="lower right")
    plt.show()
def mvp_score(arr,label):
    CHASMplus_mvp_score = 0.884424308588064

    score = metrics.roc_auc_score(label,arr)
    return score
def z_score(list1,n):
    z_list = []
    z_list = pd.Series(z_list)
    for i in range(n):
        a = list1.iloc[:, [i]]
        a_mean = a.mean(axis=0)  
          
        a_std = a.std(axis=0)  
        
        a1 = (a-a_mean)/a_std
        
        
        z_list = pd.concat([z_list,a1],axis=1)
    #print(z_list)
    return z_list
def normal(list1,n):
    nor_list = []
    nor_list = pd.Series(nor_list)
    for i in range(n):
        a = list1.iloc[:, [i]]
        maxi = a.max(axis=0)
        mini = a.min(axis=0)
        a1 = (a - mini) / (maxi - mini)
        nor_list = pd.concat([nor_list,a1],axis=1)
    return nor_list